# Load Train Test Dataset

In [1]:
import pandas as pd
import modeling.surname_data as sd
import modeling.surname_classifier as cl


df_train, df_test = sd.load_df_surnames()
df_categories = sd.load_df_categories()

# Load RNN Model

In [2]:
rnn = cl.load_model()

In [3]:
import modeling.surname_classifier as cl

cl.predict(rnn, 'james')

[[-1.3692541122436523, 14], [-1.3852684497833252, 4], [-1.5381877422332764, 0]]

# Dashbaord program

In [4]:
%%html
<style>
table.dataframe {
    width: 100%
}
iframe.wiki {
    width: 100%;
    height: 400px;
}
</style>

In [5]:
from IPython.display import display, clear_output, HTML, Javascript
import ipywidgets as widgets


def get_sample_df_surnames(category, dataset, n):
    if dataset is 'test':
        df = df_test
    else:
        df = df_train
    return df[df['category'].str.match(category)].sample(n)

def on_pick_category(category, dataset, n):
    df = get_sample_df_surnames(category, dataset, n)
    html = HTML(df.to_html(escape=False))
    return display(html)

def on_describe_dataset(dataset):
    if dataset is 'test':
        df = df_test.describe()
    elif dataset is 'train':
        df = df_train.describe()
    else:
        df = df_categories
    html = HTML(df.to_html(escape=False))
    return display(html)

## browse dataset

In [6]:
picker_w = widgets.interact(on_pick_category, category=list(df_categories['category']), dataset=['train','test'], n=5)

interactive(children=(Dropdown(description='category', options=('Arabic', 'Chinese', 'Czech', 'Dutch', 'Englis…

In [7]:
picker_w2 = widgets.interact(on_describe_dataset, dataset=['train','test', 'categies'])

interactive(children=(Dropdown(description='dataset', options=('train', 'test', 'categies'), value='train'), O…

# Predict Classification

## 1. search surname from train or test dataset

In [8]:
def on_search_surname(surname, dataset):
    if dataset is 'test':
        df_search_result = df_test[df_test['surname'].str.contains(surname, case=False)]
    else:
        df_search_result = df_train[df_train['surname'].str.contains(surname, case=False)]
    html = HTML(df_search_result.to_html(escape=False))
    return display(html)
    
    
search_w = widgets.interact(on_search_surname, surname='james', dataset=['train','test'])

interactive(children=(Text(value='james', description='surname'), Dropdown(description='dataset', options=('tr…

## 2. Predict

In [9]:
output_text = widgets.Textarea()
output_text

Textarea(value='')

In [50]:
import pprint
import json

input_text = widgets.Text()

def on_classify_surname(sender):
    result = ''
    output = cl.predict(rnn, input_text.value)
    for (prob, cat) in output:
        key = df_categories.loc[cat,'category']
        value = prob
        result = '\n'.join([result, ','.join([str(key), str(value)])])
    output_text.value = '\n'.join([input_text.value, '', '>> Infered class is ', result])

    
input_text.on_submit(on_classify_surname)
input_text

Text(value='')